## <b><font color='darkblue'>Built-in tools</font></b>
([source](https://google.github.io/adk-docs/tools/built-in-tools/)) <font size='3ptx'><b>These built-in tools provide ready-to-use functionality such as Google Search or code executors that provide agents with common capabilities.</b> For instance, an agent that needs to retrieve information from the web can directly use the `google_search` tool without any additional setup.</font>

In [2]:
from IPython.display import display, Markdown, Latex
import warnings


warnings.filterwarnings('ignore')

def show_source_code(src_path: str):
    source_code = !cat $src_path
    display(Markdown(f"""
```python
{'\n'.join(source_code)}
```"""))

### <b><font color='darkgreen'>How to Use</font></b>
([source](https://google.github.io/adk-docs/tools/built-in-tools/#how-to-use))
1. <b><font size='3ptx'>Import</font></b>: Import the desired tool from the tools module. This is `agents.tools`.
2. <b><font size='3ptx'>Configure</font></b>: Initialize the tool, providing required parameters if any.
3. <b><font size='3ptx'>Register</font></b>: Add the initialized tool to the `tools` list of your Agent.

Once added to an agent, the agent can decide to use the tool based on the `user prompt` and its `instructions`. The framework handles the execution of the tool when the agent calls it. Important: check the <b>Limitations</b> section of this page.

### <b><font color='darkgreen'>Available Built-in tools</font></b>
([source](https://google.github.io/adk-docs/tools/built-in-tools/#available-built-in-tools))

#### <b>Google Search</b>
The `google_search` tool allows the agent to perform web searches using Google Search. The `google_search` tool is only compatible with Gemini 2 models. For further details of the tool, see [**Understanding Google Search grounding**](https://google.github.io/adk-docs/grounding/google_search_grounding/).

<b><font color='orange'>Additional requirements when using the `google_search` tool</font></b><br/>
> When you use grounding with Google Search, and you receive Search suggestions in your response, you must display the Search suggestions in production and in your applications. For more information on grounding with Google Search, see Grounding with Google Search documentation for [Google AI Studio](https://ai.google.dev/gemini-api/docs/grounding/search-suggestions) or [Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/grounding/grounding-search-suggestions). The UI code (HTML) is returned in the Gemini response as `renderedContent`, and you will need to show the HTML in your app, in accordance with the policy.

In [2]:
show_source_code('builtin_tools_google_search.py')


```python
#!/usr/bin/env python
import asyncio

from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types


APP_NAME="google_search_agent"
USER_ID="user1234"
SESSION_ID="1234"

root_agent = Agent(
    name="basic_search_agent",
    model="gemini-2.0-flash",
    description="Agent to answer questions using Google Search.",
    instruction=(
        "I can answer your questions by searching the internet. "
        "Just ask me anything! For each query, please address the source."),
    # google_search is a pre-built tool which allows the agent to perform Google searches.
    tools=[google_search]
)

# Session and Runner
async def setup_session_and_runner():
  session_service = InMemorySessionService()
  session = await session_service.create_session(
      app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
  runner = Runner(
      agent=root_agent, app_name=APP_NAME, session_service=session_service)
  return session, runner


# Agent Interaction
async def call_agent_async(query):
  content = types.Content(role='user', parts=[types.Part(text=query)])
  session, runner = await setup_session_and_runner()
  events = runner.run_async(
      user_id=USER_ID, session_id=SESSION_ID, new_message=content)

  async for event in events:
    if event.is_final_response():
      final_response = event.content.parts[0].text
      print("Agent Response: ", final_response)


if __name__ == '__main__':
  asyncio.run(call_agent_async("What's the latest ai news?"))
```

In [2]:
import builtin_tools_google_search

In [3]:
response = await builtin_tools_google_search.call_agent_async(
    'What\'s the latest news about "Agent development kit"?', depress_print=True)

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


In [4]:
display(Markdown(response))

Agent Response: Here's the latest news regarding the "Agent Development Kit" (ADK):

*   **Google's ADK:** Google introduced the Agent Development Kit (ADK), an open-source framework designed to simplify the development of AI agents and multi-agent systems. It allows developers to build AI agents with fewer lines of code and streamlines integration with data platforms.
*   **Key Features of ADK:** The ADK offers precise control over agent behavior, orchestration, and is optimized for Google Cloud, with seamless integration with Gemini models and Vertex AI. It's model-agnostic, deployment-agnostic, and interoperable.
*   **Agent2Agent (A2A) Protocol:** Google Cloud is releasing native support for A2A in ADK, making it easier to build collaborative agents. This protocol enables agents from different vendors to communicate with each other, promoting a vendor-neutral standard for AI interaction.
*   **Deployment Options:** ADK provides various deployment paths for A2A-compliant agents, including Agent Engine, Cloud Run, and Google Kubernetes Engine (GKE).
*   **ADK and Agentspace:** Partners can make A2A agents available within Agentspace, Google's platform for building and managing agents.
*   **Recent Updates:** As of ADK v1.2.0+, you can deploy agents directly to Vertex AI Agent Engine with a single command.
*   **Community Support:** Over 150 organizations publicly support A2A, demonstrating the industry's need for vendor-neutral standards in agentic AI.
*   **Real-World Applications:** Companies like Tyson Foods and Gordon Food Service are using A2A systems to enhance supply chain performance and improve real-time data sharing.

#### <b>Code Execution</b>
<font size='3ptx'><b>The `built_in_code_execution` tool enables the agent to execute code, specifically when using Gemini 2 models.</b> This allows the model to perform tasks like calculations, data manipulation, or running small scripts.</font>

In [5]:
show_source_code('builtin_tools_code_execution.py ')


```python
#!/usr/bin/env python
import asyncio
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.code_executors import BuiltInCodeExecutor
from google.genai import types


AGENT_NAME = "calculator_agent"
APP_NAME = "calculator"
USER_ID = "user1234"
SESSION_ID = "session_code_exec_async"
GEMINI_MODEL = "gemini-2.0-flash"


# Agent Definition
code_agent = LlmAgent(
    name=AGENT_NAME,
    model=GEMINI_MODEL,
    code_executor=BuiltInCodeExecutor(),
    instruction=(
        "You are a calculator agent."
        "When given a mathematical expression, "
        "write and execute Python code to calculate the result."
        "Return only the final numerical result as plain text, "
        "without markdown or code blocks."
    ),
    description="Executes Python code to perform calculations.",
)

# Session and Runner
session_service = InMemorySessionService()
session = asyncio.run(session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
))
runner = Runner(
    agent=code_agent,
    app_name=APP_NAME,
    session_service=session_service)


# Agent Interaction (Async)
async def call_agent_async(query):
  content = types.Content(role="user", parts=[types.Part(text=query)])
  print(f"\n--- Running Query: {query} ---")
  final_response_text = "No final text response captured."
  try:
    # Use run_async
    async for event in runner.run_async(
        user_id=USER_ID, session_id=SESSION_ID, new_message=content
    ):
      print(f"Event ID: {event.id}, Author: {event.author}")

      # --- Check for specific parts FIRST ---
      has_specific_part = False
      if event.content and event.content.parts:
        for part in event.content.parts:  # Iterate through all parts
          if part.executable_code:
            # Access the actual code string via .code
            print(
                f"  Debug: Agent generated code:\n"
                "```python\n{part.executable_code.code}\n```")
            has_specific_part = True
          elif part.code_execution_result:
            # Access outcome and output correctly
            print(
                "  Debug: Code Execution Result: "
                f"{part.code_execution_result.outcome} - "
                f"Output:\n{part.code_execution_result.output}")
            has_specific_part = True
            # Also print any text parts found in any event for debugging
          elif part.text and not part.text.isspace():
            print(f"  Text: '{part.text.strip()}'")
            # Do not set has_specific_part=True here, as we want the final response logic below

          # --- Check for final response AFTER specific parts ---
          # Only consider it final if it doesn't have the specific code parts we just handled
          if not has_specific_part and event.is_final_response():
            if (
                event.content
                and event.content.parts
                and event.content.parts[0].text
            ):
              final_response_text = event.content.parts[0].text.strip()
              print(f"==> Final Agent Response: {final_response_text}")
            else:
              print("==> Final Agent Response: [No text content in final event]")

  except Exception as e:
    print(f"ERROR during agent run: {e}")
  print("-" * 30)


# Main async function to run the examples
async def main():
  await call_agent_async("Calculate the value of (5 + 7) * 3")
  await call_agent_async("What is 10 factorial?")


if __name__ == '__main__':
  # Execute the main async function
  try:
    asyncio.run(main())
  except RuntimeError as e:
    # Handle specific error when running asyncio.run in an already running loop (like Jupyter/Colab)
    if "cannot be called from a running event loop" in str(e):
      print("\nRunning in an existing event loop (like Colab/Jupyter).")
      print("Please run `await main()` in a notebook cell instead.")
      # If in an interactive environment like a notebook, you might need to run:
      # await main()
    else:
      raise e  # Re-raise other runtime errors
```

In [3]:
import builtin_tools_code_execution

await builtin_tools_code_execution.call_agent_async("Calculate the value of (5 + 7) * 3")

Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.



--- Running Query: Calculate the value of (5 + 7) * 3 ---


Event ID: 0801c51f-a2f8-4ea5-959a-0c04d29c2c82, Author: calculator_agent
  Debug: Agent generated code:
```python
print((5 + 7) * 3)

```
  Debug: Code Execution Result: Outcome.OUTCOME_OK - Output:
36

  Text: '36'
------------------------------


#### <b>Vertex AI Search</b>
<font size='3ptx'><b>The `vertex_ai_search_tool` uses Google Cloud's Vertex AI Search, enabling the agent to search across your private, configured data stores (e.g., internal documents, company policies, knowledge bases).</b> This built-in tool requires you to provide the specific data store ID during configuration. For further details of the tool, see [Understanding Vertex AI Search grounding](https://google.github.io/adk-docs/grounding/vertex_ai_search_grounding/).</font>

```python
import asyncio

from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
from google.adk.tools import VertexAiSearchTool

# Replace with your actual Vertex AI Search Datastore ID
# Format: projects/<PROJECT_ID>/locations/<LOCATION>/collections/default_collection/dataStores/<DATASTORE_ID>
# e.g., "projects/12345/locations/us-central1/collections/default_collection/dataStores/my-datastore-123"
YOUR_DATASTORE_ID = "YOUR_DATASTORE_ID_HERE"

# Constants
APP_NAME_VSEARCH = "vertex_search_app"
USER_ID_VSEARCH = "user_vsearch_1"
SESSION_ID_VSEARCH = "session_vsearch_1"
AGENT_NAME_VSEARCH = "doc_qa_agent"
GEMINI_2_FLASH = "gemini-2.0-flash"

# Tool Instantiation
# You MUST provide your datastore ID here.
vertex_search_tool = VertexAiSearchTool(data_store_id=YOUR_DATASTORE_ID)

# Agent Definition
doc_qa_agent = LlmAgent(
    name=AGENT_NAME_VSEARCH,
    model=GEMINI_2_FLASH, # Requires Gemini model
    tools=[vertex_search_tool],
    instruction=f"""You are a helpful assistant that answers questions based on information found in the document store: {YOUR_DATASTORE_ID}.
    Use the search tool to find relevant information before answering.
    If the answer isn't in the documents, say that you couldn't find the information.
    """,
    description="Answers questions using a specific Vertex AI Search datastore.",
)

# Session and Runner Setup
session_service_vsearch = InMemorySessionService()
runner_vsearch = Runner(
    agent=doc_qa_agent, app_name=APP_NAME_VSEARCH, session_service=session_service_vsearch
)
session_vsearch = session_service_vsearch.create_session(
    app_name=APP_NAME_VSEARCH, user_id=USER_ID_VSEARCH, session_id=SESSION_ID_VSEARCH
)

# Agent Interaction Function
async def call_vsearch_agent_async(query):
    print("\n--- Running Vertex AI Search Agent ---")
    print(f"Query: {query}")
    if "YOUR_DATASTORE_ID_HERE" in YOUR_DATASTORE_ID:
        print("Skipping execution: Please replace YOUR_DATASTORE_ID_HERE with your actual datastore ID.")
        print("-" * 30)
        return

    content = types.Content(role='user', parts=[types.Part(text=query)])
    final_response_text = "No response received."
    try:
        async for event in runner_vsearch.run_async(
            user_id=USER_ID_VSEARCH, session_id=SESSION_ID_VSEARCH, new_message=content
        ):
            # Like Google Search, results are often embedded in the model's response.
            if event.is_final_response() and event.content and event.content.parts:
                final_response_text = event.content.parts[0].text.strip()
                print(f"Agent Response: {final_response_text}")
                # You can inspect event.grounding_metadata for source citations
                if event.grounding_metadata:
                    print(f"  (Grounding metadata found with {len(event.grounding_metadata.grounding_attributions)} attributions)")

    except Exception as e:
        print(f"An error occurred: {e}")
        print("Ensure your datastore ID is correct and the service account has permissions.")
    print("-" * 30)

# --- Run Example ---
async def run_vsearch_example():
    # Replace with a question relevant to YOUR datastore content
    await call_vsearch_agent_async("Summarize the main points about the Q2 strategy document.")
    await call_vsearch_agent_async("What safety procedures are mentioned for lab X?")

# Execute the example
# await run_vsearch_example()

# Running locally due to potential colab asyncio issues with multiple awaits
try:
    asyncio.run(run_vsearch_example())
except RuntimeError as e:
    if "cannot be called from a running event loop" in str(e):
        print("Skipping execution in running event loop (like Colab/Jupyter). Run locally.")
    else:
        raise e
```

#### <b>BigQuery</b>
These are a set of tools aimed to provide integration with BigQuery...([more](https://google.github.io/adk-docs/tools/built-in-tools/#bigquery))

### <b><font color='darkgreen'>Use Built-in tools with other tools</font></b>
([source](https://google.github.io/adk-docs/tools/built-in-tools/#use-built-in-tools-with-other-tools)) <b><font size='3ptx'>The following code sample demonstrates how to use multiple built-in tools or how to use built-in tools with other tools by using multiple agents:</font></b>

```python
from google.adk.tools import agent_tool
from google.adk.agents import Agent
from google.adk.tools import google_search
from google.adk.code_executors import BuiltInCodeExecutor


search_agent = Agent(
    model='gemini-2.0-flash',
    name='SearchAgent',
    instruction="""
    You're a specialist in Google Search
    """,
    tools=[google_search],
)
coding_agent = Agent(
    model='gemini-2.0-flash',
    name='CodeAgent',
    instruction="""
    You're a specialist in Code Execution
    """,
    tools=[BuiltInCodeExecutor],
)
root_agent = Agent(
    name="RootAgent",
    model="gemini-2.0-flash",
    description="Root Agent",
    tools=[agent_tool.AgentTool(agent=search_agent), agent_tool.AgentTool(agent=coding_agent)],
)
```

### <b><font color='darkgreen'>Limitations</font></b>

#### <font color='orange'><b>Warning1</b></font>
> Currently, for each root agent or single agent, only one built-in tool is supported. No other tools of any type can be used in the same agent.

For example, the following approach that uses a built-in tool along with other tools within a single agent is not currently supported:
```python
root_agent = Agent(
    name="RootAgent",
    model="gemini-2.0-flash",
    description="Root Agent",
    tools=[custom_function, BuiltInCodeExecutor], # <-- BuiltInCodeExecutor not supported when used with tools
)
```

#### <b><font color='orange'>Warning2</font></b>
> Built-in tools cannot be used within a sub-agent.

For example, the following approach that uses built-in tools within sub-agents is not currently supported:
```python
search_agent = Agent(
    model='gemini-2.0-flash',
    name='SearchAgent',
    instruction="""
    You're a specialist in Google Search
    """,
    tools=[google_search],
)
coding_agent = Agent(
    model='gemini-2.0-flash',
    name='CodeAgent',
    instruction="""
    You're a specialist in Code Execution
    """,
    tools=[BuiltInCodeExecutor],
)
root_agent = Agent(
    name="RootAgent",
    model="gemini-2.0-flash",
    description="Root Agent",
    sub_agents=[
        search_agent,
        coding_agent
    ],
)
```

## <b><font color='darkblue'>Third Party Tools</font></b>
([source](https://google.github.io/adk-docs/tools/third-party-tools/#third-party-tools)) <font size='3ptx'><b>ADK is designed to be highly extensible, allowing you to seamlessly integrate tools from other AI Agent frameworks like [CrewAI](https://www.crewai.com/) and [LangChain](https://python.langchain.com/docs/introduction/).</b> This interoperability is crucial because it allows for faster development time and allows you to reuse existing tools.</font>

### <b><font color='darkgreen'>1. Using LangChain Tools</font></b>
<b><font size='3ptx'>ADK provides the <font color='blue'>LangchainTool</font> wrapper to integrate tools from the [LangChain](https://python.langchain.com/docs/introduction/) ecosystem into your agents.</font></b>

#### <b>Example: Web Search using LangChain's Tavily tool</b>
[**Tavily**](https://tavily.com/) provides a search API that returns answers derived from real-time search results, intended for use by applications like AI agents.

1. <b><font size='3ptx'>Install Dependencies</font></b>: Ensure you have the necessary [**LangChain**](https://python.langchain.com/docs/introduction/) packages installed. For example, to use the Tavily search tool, install its specific dependencies:

In [4]:
#!pip install langchain_community tavily-python
!pip freeze | grep -P "(langchain|tavily)"

langchain==0.3.27
langchain-community==0.3.27
langchain-core==0.3.72
langchain-google-genai==2.1.6
langchain-mcp-adapters==0.0.7
langchain-openai==0.3.12
langchain-text-splitters==0.3.9
tavily-python==0.7.10


2. Obtain a [**Tavily**](https://tavily.com/) API KEY and export it as an environment variable.

```shell
export TAVILY_API_KEY=<REPLACE_WITH_API_KEY>
```

3. <b><font size='3ptx'>Import</font></b>: Import the `LangchainTool` wrapper from ADK and the specific `LangChain` tool you wish to use (<font color='brown'>e.g, `TavilySearchResults`</font>).
```python
from google.adk.tools.langchain_tool import LangchainTool
from langchain_community.tools import TavilySearchResults
```

4. <b><font size='3ptx'>Instantiate & Wrap</font></b>: Create an instance of your [**LangChain**](https://python.langchain.com/docs/introduction/) tool and pass it to the `LangchainTool` constructor.
```python
# Instantiate the LangChain tool
tavily_tool_instance = TavilySearchResults(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True,
)

# Wrap it with LangchainTool for ADK
adk_tavily_tool = LangchainTool(tool=tavily_tool_instance)
```

5. <b><font size='3ptx'>Add to Agent</font></b>: Include the wrapped LangchainTool instance in your agent's `tools` list during definition.
```python
from google.adk import Agent

# Define the ADK agent, including the wrapped tool
my_agent = Agent(
    name="langchain_tool_agent",
    model="gemini-2.0-flash",
    description="Agent to answer questions using TavilySearch.",
    instruction="I can answer your questions by searching the internet. Just ask me anything!",
    tools=[adk_tavily_tool] # Add the wrapped tool here
)
```

Here's the full code combining the steps above to create and run an agent using the LangChain Tavily search tool:

In [5]:
show_source_code('thirdparty_tools_langchain_travily_search.py')


```python
#!/usr/bin/env python
import asyncio
import os

from google.adk import Agent, Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.langchain_tool import LangchainTool
from google.genai import types
from langchain_tavily import TavilySearch


TAVILY_API_KEY = None
# Ensure TAVILY_API_KEY is set in your environment
if not (TAVILY_API_KEY := os.getenv("TAVILY_API_KEY")):
  raise Exception("Error: TAVILY_API_KEY environment variable not set.")

print(f'TRAVILY_API_KEY: {"*" * len(TAVILY_API_KEY)}')


APP_NAME = "news_app"
USER_ID = "1234"
SESSION_ID = "session1234"


# Instantiate LangChain tool
tavily_search = TavilySearch(
    max_results=5,
    search_depth="advanced",
    include_answer=True,
    include_raw_content=True,
    include_images=True)


# Wrap with LangchainTool
adk_tavily_tool = LangchainTool(tool=tavily_search)


# Define Agent with the wrapped tool
my_agent = Agent(
    name="langchain_tool_agent",
    model="gemini-2.0-flash",
    description="Agent to answer questions using TavilySearch.",
    instruction=(
        "I can answer your questions by searching the internet. "
        "Just ask me anything!"),
    tools=[adk_tavily_tool] # Add the wrapped tool here
)


async def setup_session_and_runner():
  session_service = InMemorySessionService()
  session = await session_service.create_session(
      app_name=APP_NAME,
      user_id=USER_ID,
      session_id=SESSION_ID)
  runner = Runner(
      agent=my_agent,
      app_name=APP_NAME,
      session_service=session_service)

  return session, runner


# Agent Interaction
async def call_agent_async(query):
  content = types.Content(role='user', parts=[types.Part(text=query)])
  session, runner = await setup_session_and_runner()
  events = runner.run_async(
      user_id=USER_ID, session_id=SESSION_ID, new_message=content)

  async for event in events:
    if event.is_final_response():
      final_response = event.content.parts[0].text
      print("Agent Response: ", final_response)


if __name__ == '__main__':
  asyncio.run(call_agent_async('Stock price of GOOG'))
```

In [1]:
import thirdparty_tools_langchain_travily_search

await thirdparty_tools_langchain_travily_search.call_agent_async('Stock price of GOOG')

/usr/local/google/home/johnkclee/Github/ml_articles/env/lib/python3.12/site-packages/pydantic/_internal/_fields.py:198: UserWarning: Field name "config_type" in "SequentialAgent" shadows an attribute in parent "BaseAgent"
  warnings.warn(
Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


TRAVILY_API_KEY: *************************************


Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


Agent Response:  The stock price for Alphabet Inc. (GOOG) is 201.63 USD as of the latest closing time on August 11, 2025.


### <b><font color='darkgreen'>2. Using CrewAI tools</font></b>
ADK provides the CrewaiTool wrapper to integrate tools from the CrewAI library. ([more](https://google.github.io/adk-docs/tools/third-party-tools/#2-using-crewai-tools))